In [ ]:
import random
from pathlib import Path

import polars as pl

from imdataset_creator.datarules import DatasetBuilder
from imdataset_creator.datarules import Rule

In [ ]:
filename = "filedb.arrow"
df = pl.read_ipc(filename, memory_map=False)
df

In [ ]:
# shuffle dataframe
def shuffle(df: pl.DataFrame, rand_col="rnd") -> pl.DataFrame:
    return df.with_columns(pl.Series(rand_col, [random.random() for _ in range(len(df))])).sort(rand_col).drop(rand_col)
df = shuffle(df)

In [ ]:
# delete random items in dataframe
thresh = 0.95
import random


def rnd(_):
    return random.random()


def drop_rand(df: pl.DataFrame, exclude: list[str], thresh: float = 0.9) -> pl.DataFrame:
    new = df.select(pl.when(pl.all().exclude(*exclude).map_elements(rnd, skip_nulls=False) < thresh).then(pl.all()))
    return new.with_columns(df.select(*exclude)).select(df.columns)


df = drop_rand(df, ["path"], 0.5)

In [ ]:
# save
df.rechunk().write_ipc(filename)

In [ ]:
from polars import col
df.select(col("modifiedtime") == col("modifiedtime").max())

### plot resolution vs modified time

In [ ]:
plt.figure(figsize=(5, 10), dpi=300)
plt.scatter(
    x=df.select("modifiedtime"),
    y=df.with_columns(sum_res=pl.col("resolution").apply(lambda lst: sum(lst))).select("sum_res"),
    s=0.05,
    c=df.select("checkedtime"),
    alpha=0.5
)
plt.xlabel("modifiedtime")
plt.ylabel("sum resolution")